### What is counterfactual?

$\textbf{Counterfactual Condition (CF)}$ is a conditional containing if-clause which is contrary to the fact.
Examples:
1. If Oswald $\textbf{did not shoot}$ Kennedy, then someone else $\textbf{did} $
2. If I $\textbf{hadn’t taken a sip}$ of this hot coffee, I $\textbf{wouldn’t have burned}$ my tongue

Basically, it's an expression of form: If $\textbf{CAUSE}$, then $\textbf{EVENT}$. 

Further, we will be calling **counterfactual** an object, where **CAUSE** is true, s.t. **EVENT** happened. Examples:
1. "If this cat didn't have whiskers, then it would be recognized by classifier as a dog.". Counterfacutal in this case is "cat without whiskers", which consequently recognized as a dog.
2. "If I've studied more, I would land this job". Counterfactual in this case is "person who studied more", which consequently landed this job.

Why counterfactials are important? They are the answers to questions of type "Why event P happened?". According to Miller 2019 et al, if we want to understand why event $P$ happened, we don't usually ask "why P happened". We rather ask why $P$ happened instead of some other event $Q$, even when $Q$ is implicit. So, people are interested in answering ''Why P happened instead of Q?'', even when Q is implicit. 

Examples:
1. Why are you so late today? (implicitly: why didn't you come earlier?)
2. Why classifier gave this text "this is not bad" negative sentimental? (implicitly: why not positive sentiment?)

So, when we have a question of type "Why event **P** happened, instead of **Q**?", we answer it as counterfactual "If **CAUSE**, then **Q** would've happened"

### Why counterfactuals are helpful in Machine Learning?

As mentioned before, counterfactuals are helpful to get answers for questions of type "Why event **P** happened instead of **Q**?". One way to ask such question in Machine Learning context is "Why this particular object was classified as class **A**, not **B**?". The counterfactual in this case would be a modified object, which has class **B**

By generating counterfactuals we can get better understanding of our models, e.g. discover biases in dataset.

Let's consider an example:

<img src="data/cat_1.jpg" width=200>

Say, our classifier recognized this image as 'Egyptian Cat' with probability 34.6%. 

Now, we can ask a question *"Why classifier recognized image as 'Egyptian Cat' with such high probability, why not lower probability?"*. And we can get a counterfactual answer *"If this cat didn't have whiskers, then its probability would be below 1%"*. This clearly indicates that at least for this particular image classifier bases its decision heavily on cat having whiskers.

We can test it for other cat images and if we find out that decision about them being a cat also heavily based on existence of whiskers, we might doubt our classifier that it's really only learned about cat having whiskers. It might be either what we really want (because we know that in the future only cats will have whiskers) or what we want to avoid (e.g. rats also have whiskers and they might appear in the test dataset).

There is a research paper [LA Hendricks et al 2018](https://arxiv.org/abs/1806.09809) which is generating natural language explanations just like those in the example. This approach requires dataset which is labeled with descriptions of images, which makes this approach limited to having augmented dataset. That's why we're not considering it here.

Another approach of generating counterfactual is by producing a variation of image, where some parts are obfuscated (e.g. masked / blurred / etc). Let's consider the following image:

<img src="data/cat_dog.png" width=300>

Probability of the dog on this image is 29% according to classifier. But it's less than 1% for the image, where left eye of the dog masked.

<img src="data/cat_dog_masked.jpg" width=300>

### How to generate counterfactual?
The easiest way (not the best) to generate counterfactual is by changing pixels s.t. new picture will be classified as a different class.

Let's denote 
1. $x_0$ - input image.
2. $P_i(x)$ - output (probability) of the class $i$ for image $x$ (e.g. could be the i-th neuron on the last layer of neural network).

Then optimization problem problem is:

$$
\min_{x}{P_i(x) + \lambda ||x-x_0||}
$$

We are generating new image by reducing probability of class $i$, while staying close to the original image $x_0$. Hyperparameter $\lambda$ controls the tradeoff between closeness to $x_0$ and how low probability $P_i(x)$ is. So, this problem can be solved by using gradient descent.

Below is the result of optimizing this function using gradient descent (100 iterations, gradient step=1.0, $\lambda=1$) [code](https://github.com/destinityx2/ml-explain/blob/master/mlexplain/counterfactuals/vanilla_gradient_counterfactual.py):

<img src="data/cat_original_counterfactual_diff.png" width=1000 />

Here we can see that the difference is almost indistinguishable. Original image has probability $34.7\%$ of this image being an 'Egyptian Cat', while counterfactual has $0.2\%$ probability.

It's important to note here that this kind of counterfactual isn't very useful in a sense of understanding what is important for classifier to make its decision, because we don't really see where classifier looks at in particular. Next section discusses what makes a good counterfactual.

### What makes a good counterfactual?
Let's consider an example:

<img src="data/cat_1.jpg" width=200>

We can ask different questions to get better understanding about the decision made by classifier:
1. Why this image has been classified as Egyptian cat (most probable class), and not some other class?
2. This image has been classified as Egyptian cat with $34.6\%$ confidence, why not lower probability?
3. This image has been classified as Egyptian cat, why not Tabby cat?

All these questions could be answered with counterfactuals:
1. If image didn't contain whiskers, then it would be classified as other class (tiger_cat).
2. If image didn't have cat in it, but coyote, then Egyptian cat class would have lower probability $(\sim0\%)$.
3. If this cat had green eyes instead of blue eyes, it would be classified as Tabby Cat.

As can be seen from previous examples, there are plenty of questions that could be asked and there are plenty of answers that we can get. Indeed, one question can have a lot of different answers and most of them are not very practical (e.g. we can reduce probability of Egyptian cat by making all pixels to be grey, but it doesn't really help us to get more insights about why particular decision has been made). According to [Miller 2019 et al](https://arxiv.org/abs/1706.07269?context=cs), there are conditions which counterfactuals need to satisfy in order to be more useful:
1. *Pragmatic goals*. We need to think about what goal asking person is aiming to achieve. Examples:
    1. *Identify bias in the dataset*. E.g. if all animals with blue eyes are classified as Egyptian cat, explanation of form "Image classified as Egyptian cat, because it has blue eyes" might be useful.
    2. *Identify most important parts of the image to understand where classifier looks at*. For this problem we can produce an image where some parts are hidden, s.t. probability of the target class is significantly reduced.
    3. *Identify datapoints used during training which made classifier to make this particular decision*.
    4. *Detect if classifier is biased against gender, race, etc*. For example, if we have credit scoring classifier (whether to grant credit or not), then if we ask question "Why this person didn't get credit?" and we get answer "If this person had different race / gender, then he would've gotten credit" clearly identifies bias. Important to note here that if we get such an answer, then there is bias for sure, but if we don't, then it doesn't mean there is no bias. 
    5. *Detect which neural network units were the most responsible for making this decision*.

2. *Explanation should be minimal*. If we make perturbation to image and some distance is defined, then we would prefer counterfactuals with smaller distance. It's important to note that distance could be defined differently. For example, one might argue that changing color of only one eye has smaller distance that coloring both eyes, but another might argue differently (because we simply don't see many cats with eyes of different colors). 

So, in order to generate counterfactual we need to alternate some object, that will lead to the desired outcome. In typical Machine Learning setting we have 4 objects: dataset, model, learning algorithm and input object to be classified. One can come up with even more objects which we can alternated, e.g. data gathering process / data sources. Also, for generating useful counterfactual this alternation should be minimal in terms of distance.

### Problem formulation
Suppose we have:
1. Set of objects $\mathcal{X}$ (e.g. set of all possible images of size $224x224x3$)
2. Criteria $C(x): \mathcal{x} \rightarrow Boolean$ (e.g. $C(x) = 1$, if $x$ classified as class $A$, $C(x) = 0$ otherwise)
3. Distance $d(x, y)$ (e.g. number of pixels which differ between two images)
4. Input object $x_0 \in \mathcal{X}$

Goal is to find $x^*$:
$$ 
x^* = arg\min_{x \in \mathcal{X} \& C(x)=1} d(x_0, x)
$$

In this problem set of objects $\mathcal{X}$ defines set of alternations to input object $x_0 \in \mathcal{X}$. Criteria $C(x)$ determines whether condition of $x$ being counterfactual is satisfied or not.

Note that this problem is abstract. We can make this problem concrete by specifying the following objects: $\mathcal{X}, C(x), d(x, y), x_0$. Examples of concrete formulations:
1. $\mathcal{X}$ = $\{$set of all possible neural network architectures with input $224x224x3$ and 1000 output classes$\}$; $x_0$ = vgg19; $d(x, y) = $ number of units different between x and y. $C(x) = 1$ iff for input dataset $Y_0$ classifier $x$ has 0 error rate for images of class 'Tiger Cat'.
2. $\mathcal{X}$ = $\{$ set of all possible images of size $224x224x3$ $\}$; $x_0$ = cat image from example above; $d(x,y) =$ number of pixels different between $x$ and $y$. $C(x) = 1$ iff probability of $x < p_0$.
3. $\mathcal{X}$ = $\{$ set of all possible modifications of $x_0$ image, where only eye pixels are changed $\}$; $x_0$ = cat image from example above; $d(x,y) =$ number of pixels different between $x$ and $y$. $C(x) = 1$ iff probability of $x < p_0$.
3. $\mathcal{X}$ = $\{$ modification of sentence $x_0$, where some words are removed $\}$; $x_0$ = "This product is not bad"; $d(x,y) = | |x| - |y| |$ - different between length of $x$ and $y$. $C(x) = 1$ iff  $x$ classified as "negative review" instead of "positive review".

### Notes
There are different methods for understanding NNs better.
We can find approximate purpose of each convolutional layer by using DeconvNets (Zeiler & Fergus, 2014). Also, by using gradient descent on input image we can find what activates certain inputs.

In this work we approach problem of understanding NN from a different perspective: by generating counterfactuals. 

Generating natural adversaries proposes to search for an element in z-space